<a href="https://colab.research.google.com/github/yn2a75/benkyokai/blob/main/pdf2text_ExtractCompanyName_RenameFle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
#必要なライブラリをインストール
! pip install pdfminer.six
! pip install pandas
! pip install openpyxl
! pip install xlwt
! pip install xlsxwriter

### PDFファイルからテキストを出力する関数を作成

In [130]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import os
from io import StringIO

#PDFからテキストを出力する関数を定義
def convert_pdf2txt(filename):
  pdf_file = os.path.join(pdffiles_folder, filename)
  #バイナリファイルを読み込むためにファイルを開く('rb'を指定)
  fp = open(pdf_file, 'rb')
    
  outfp = StringIO()
        
  rmgr = PDFResourceManager()
  lprms = LAParams()
  device = TextConverter(rmgr, outfp, laparams=lprms)
  iprtr = PDFPageInterpreter(rmgr, device)
    
  for page in PDFPage.get_pages(fp):
    iprtr.process_page(page)

#テキスト内容を取得し、改行とスペースを削除
  text = outfp.getvalue()
  text=text.replace("\n", "")
  text=text.replace("   ", "\n　")
  text=text.replace(" ", "")

#テキスト内容を返す
  return text

In [131]:
#GoogleDriveをマウント
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [132]:
#PDFを保管しているフォルダのpathを指定

pdffiles_folder=r"/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/PDFfiles"

#テキストを格納する空のリストを作成
text_list=[]


filelist = os.listdir(pdffiles_folder)
filelist = [f for f in filelist if '.pdf' in f]
pdfpaths = [os.path.join(pdffiles_folder,f) for f in filelist if '.pdf' in f]

#リストにPDFのテキストを追加
for filename in filelist:
  text_list.append(convert_pdf2txt(filename))

In [133]:
len(text_list)
len(filelist)
len(pdfpaths)
text_list

['各\u3000位2022年3月10日上場会社名代表者（コード番号問合せ先責任者取締役経営管理本部長及川\u3000光広03-6262-9877）（TEL株式会社大盛工業代表取締役社長福井\u3000龍一1844）業績予想の修正に関するお知らせ最近の業績動向を踏まえ、2021年９月14日に公表した業績予想を下記の通り修正いたしましたのでお知らせいたします。●業績予想の修正について2022年7月期第2四半期（累計）連結業績予想数値の修正（2021年8月1日～2022年1月31日）売上高営業利益経常利益親会社株主に帰属する四半期純利益1株当たり四半期純利益記百万円901233236.4244百万円4512984186.2185百万円2,2282,200△28△1.32,362百万円1,4781,64817011.51,919百万円871233641.9242百万円3212087269.0181百万円66932640.1191百万円2710678284.61442022年7月期第2四半期（累計）個別業績予想数値の修正（2021年8月1日～2022年1月31日)売上高営業利益経常利益四半期純利益1株当たり四半期純利益前回発表予想(A)今回修正予想(B)増減額(B-A)増減率(%)（ご参考）前期第2四半期実績（2021年7月期第2四半期）前回発表予想(A)今回修正予想(B)増減額(B-A)増減率(%)（ご参考）前期第2四半期実績（2021年7月期第2四半期）修正の理由2022年7月期第２四半期(累計)の連結業績予想は、売上高につきましては、建設事業の新規工事の受注時期が計画よりも遅れたものの、概ね計画通りの計上となる見込みです。営業利益につきましては、収益性の高い工事の増額変更により、工事利益の上積みが行えたことで32百万円増加し、123百万円の見込みです。経常利益及び親会社株主に帰属する四半期純利益は、完成工事利益の増加等により、経常利益123百万円、親会社株主に帰属する四半期純利益93百万円の見込みです。また、個別業績につきましても、公表しました業績予想との差異が発生しましたことから、あわせて開示をいたします。なお、第２四半期(累計)においては、完成工事利益の上積みにより計画を上回る結果となりましたが、通期としましては新型コロナウィルスの感染拡大等、先行きの不透明な状況

### テキストから会社名を抜き出す
ginzaとspacyを使う

In [134]:
#ライブラリをインストール
!pip install -U ginza
!pip install -U ginza ja-ginza

In [135]:
#spacyをインポートしてインスタンス化
import spacy
from spacy import displacy
nlp = spacy.load('ja_ginza')

OrderDictを使ってリストから、順序を維持したうえで重複を削除したリストを作成する
https://www.delftstack.com/ja/howto/python/remove-duplicates-from-list-python/

In [136]:
#OrderDictをインポート
from collections import OrderedDict

### テキストから会社名を返す関数を作成

In [137]:
#テキストから会社名を返す関数をつくる

def Text2CompanyName(text):
  doc=nlp(text)
  #空のリストを準備
  company_names_=[]
#固有表現ラベルがCompanyをリストに追加
  for ent in doc.ents: 
    if ent.label_ =='Company':
      #print(ent.text, ent.label_)
      company_names_.append(ent.text) 
#      company_names=list(set(company_names_))
      company_names=list(OrderedDict.fromkeys(company_names_))
      #StrCompanyName = "".join(company_names)
  if len(company_names_)==0:
    return ["なし"]
  else:
    return company_names

### 参考
listが空かどうか判定する方法
https://qiita.com/yonedaco/items/d0f65ca3dad2e085a51d

### 用意した関数を使い、テキストから会社名を抽出して新たなリストを作成する

In [138]:
#companyの名前を入れる空のリストを準備
companyNames1=[]
for company in text_list:
    companyNames1.append(Text2CompanyName(company))

### 2次元リストの各行の0番目値を取得して新たにリストを作る  
https://ja.stackoverflow.com/questions/47331/python%E3%81%A7%E4%BA%8C%E6%AC%A1%E5%85%83%E9%85%8D%E5%88%97%E3%81%AE%E4%B8%AD%E3%81%AE%E5%90%84%E8%A6%81%E7%B4%A0%E3%81%AEn%E7%95%AA%E7%9B%AE%E3%81%A0%E3%81%91%E3%82%92%E5%8F%96%E3%82%8A%E5%87%BA%E3%81%97%E3%81%A6-%E8%A6%81%E7%B4%A0%E3%81%A8%E3%81%97%E3%81%A6%E4%B8%A6%E3%81%B9%E3%81%9F%E3%81%84

In [139]:
#CompanyName1の各行の0番目の値をCompanyNames2に格納
companyNames2=list(map(lambda companyName1:companyName1[0],companyNames1))

### pandasでエクセル表のもととなる一覧表を作成
https://www.delftstack.com/ja/howto/python-pandas/pandas-create-dataframe-from-list/

In [140]:
#pandasをインポート
import pandas as pd

In [141]:
#データフレームを作成
df=pd.DataFrame(list(zip(pdfpaths,filelist,text_list,companyNames1,companyNames2)),columns=["path","filename","text","companynames","companyname"])

In [142]:
df

,path,filename,text,companynames,companyname
0,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220302598726.pdf,各 位2022年3月10日上場会社名代表者（コード番号問合せ先責任者取締役経営管理本部長及川...,[なし],なし
1,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220309502942.pdf,各位2022年3月10日会社名\n 天馬株式会社代表者名代表取締役社長廣野裕彦（コード：79...,[有限公司],有限公司
2,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220309502571.pdf,2022年３月10日\n \n \n \n \n \n \n \n 代表者名\n \n 代表...,"[伊藤忠エネクス株式会社, コーポレート・コミュニケーション室]",伊藤忠エネクス株式会社
3,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503726.pdf,2022年３月10日\n 会\n 社\n 名\n \n 株式会社シマノ代表者名\n \n 代...,"[株式会社シマノ, 定時株主総会, Singapore)Pte.Ltd.]",株式会社シマノ
4,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220309503096.pdf,2022年4月期第3四半期決算短信〔日本基準〕（連結）2022年3月10日上場取引所東（氏名...,"[株式会社テンポスホールディングスコード番号2751, 株式会社テンポスホールディングス, ...",株式会社テンポスホールディングスコード番号2751
5,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503194.pdf,各\n 位2022年3月10日会社名\n 大豊建設株式会社代表者名代表取締役社長\n 大隅健...,"[東京土木支店副支店長, 東京土木支店副支店長東京土木支店副支店長, 東北支店副支店長, 東...",東京土木支店副支店長
6,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503662.pdf,各位\n \n 2022年３月10日会社名テ代表者名代表取締役社長ラ株式会真\n 船\n 達...,"[定時株主総会, HLBMeisei有限責任監査法人, HLBMeisei]",定時株主総会
7,/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/P...,140120220310503389.pdf,2022年10月期第1四半期決算説明資料トビラシステムズ株式会社（東証一部4441） 1事業...,"[株式会社（東証一部4441, 会社概要, トビラシステムズ株式会社, TalkBook, ...",株式会社（東証一部4441


In [143]:
#データフレームをエクセル出力
output_excel_path=pdffiles_folder + "list.xlsx"
df.to_excel(output_excel_path , engine='xlsxwriter')

### Pandasでエクセルファイルに書き出そうとすると表示される「IllegalCharacterError」を回避
pandasの書き出しエンジンをxlsxwriterに変更する  
df.to_excel("test.xlsx", engine='xlsxwriter')  
https://karupoimou.hatenablog.com/entry/2019/09/29/235528

### PDFのファイル名を変更し、別フォルダに移動して保存する。
https://note.nkmk.me/python-shutil-move/

In [144]:
#会社名を確認したエクセルをデータフレームに読み込む
df2 = pd.read_excel(output_excel_path)

In [145]:
#移動先のフォルダを指定
#strtodir=pdffiles_folder +"\\" + "processed"
strtodir='/content/drive/MyDrive/その他/PDFのOCR処理とファイル名変更/PDFfiles/processed'

In [146]:
#新しいファイル名を含めた移動先のパスを指定
df2["topath"]=strtodir + "/" +df2["companyname"] + df2["filename"] 

In [147]:
#ファイルを移動するためのモジュールをインポート
import shutil
import os

In [148]:
#if文でファイルが存在するかを確認した上で、ファイルを移動する
for path, topath in zip(df2["path"],df2["topath"]):
    if os.path.exists(path):
        new_path=shutil.move(path,topath)
    else:
        print("file in not exist")
#    print(path,topath)